# The Final 

In [4]:
#Librerias. 
import numpy as np 
import pandas as pd

import sklearn as sk  
import seaborn as sns

import matplotlib.pyplot as plt

import opendatasets as od
import matplotlib.dates as mdates
import scipy.stats as stats

from scipy.stats import genpareto, norm, gaussian_kde

from scipy.stats import anderson, levene, bartlett, gumbel_r, genextreme
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

import ruptures as rpt 

In [26]:
###Vamos a utilizar base de datos simuladas para obtener el tema de las diferencias

df= pd.read_excel('insurance/foi-data-31Oct19.xlsx',sheet_name='Settled Claims')

fig = go.Figure()

####Añadir la línea de Reclamaciones
fig.add_trace(go.Scatter(x=df['Payment Date'], y=df[' Amount'], mode='lines', name='No. the Reclamaciones'))

###Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=20  # ajusta este valor según la cantidad de etiquetas de fecha que quieras mostrar
)

###Añadir títulos y etiquetas
fig.update_layout(
    title='Historial de Ahorro del usuario-emprendimiento',
    xaxis_title='Fecha',
    yaxis_title='Monto Total Ahorrado',
    legend_title='Leyenda',
    width=1000,
    height=600
)
fig.show()

In [27]:
# Número de divisiones que quieres
n = 15  # Puedes ajustar este valor

# Definir el tamaño de cada segmento
segment_size = len(df) // n

# Crear una lista para almacenar los subconjuntos en orden descendente
subsets = []

# Crear los subconjuntos en orden descendente y almacenarlos en la lista
for i in range(n):
    start_idx = i * segment_size
    if i < n - 1:
        end_idx = (i + 1) * segment_size
    else:
        end_idx = len(df)  # Asegura que el último segmento llegue hasta el final
    
    # Crear subconjunto y añadirlo al principio de la lista (para orden descendente)
    subset = df.iloc[start_idx:end_idx]
    subsets.insert(0, subset)  # Usamos insert(0, ...) para que el subconjunto más reciente esté al inicio

# Crear la figura
fig = go.Figure()

# Añadir trazos para cada subconjunto, con numeración descendente
for i, subset in enumerate(subsets):
    # Añadir la línea del subconjunto correspondiente, con numeración descendente
    fig.add_trace(go.Scatter(x=subset['Payment Date'], 
                             y=subset[' Amount'], 
                             mode='lines', 
                             name=f'Segmento {i+1}'))  # La numeración ahora es ascendente en el orden visual

# Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=20  # ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

# Añadir títulos y etiquetas
fig.update_layout(
    title='Historial del Fondo por segmentos de tiempo',
    xaxis_title='Fecha',
    yaxis_title='Monto Total ahorrado',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

In [28]:
# Ahora, vamos a calcular las diferencias y graficarlas en una sola gráfica
fig = go.Figure()

# Añadir trazos para las diferencias de cada subintervalo
for i,subset in enumerate(subsets):
    # Calcular las diferencias consecutivas
    diff_values = subset[' Amount'].diff().dropna()
    
    # Añadir el trazo para las diferencias en el subintervalo
    fig.add_trace(go.Scatter(x=subset['Payment Date'][1:], y=diff_values,
                             mode='lines', name=f'Diferencias Subintervalo {i+1}'))

# Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10  # Ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

# Añadir títulos y etiquetas
fig.update_layout(
    title='Gráfico de Diferencias de Ingresos - Gastos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

# Mostrar el gráfico
fig.show()

In [29]:
import numpy as np

#Vamos a calcular las diferencias y los excesos sobre los percentiles 90 y 10
fig = go.Figure()

#Añadir trazos para las diferencias de cada subintervalo
for i, subset in enumerate(subsets):
    ####Calcular las diferencias consecutivas
    diff_values = subset[' Amount'].diff().dropna()
    
    ####Calcular los percentiles 90 y 10 del subintervalo
    threshold_90 = np.percentile(diff_values, 90)
    threshold_10 = np.percentile(diff_values, 6)
    
    ####Identificar los valores que exceden el percentil 90 (excesos positivos)
    excess_positive_values = diff_values[diff_values > threshold_90]
    excess_positive_dates = subset['Payment Date'][1:][diff_values > threshold_90]  # Ajustar las fechas
    
    ####Identificar los valores que caen por debajo del percentil 10 (excesos negativos)
    excess_negative_values = diff_values[diff_values < threshold_10]
    excess_negative_dates = subset['Payment Date'][1:][diff_values < threshold_10]  # Ajustar las fechas

    ####Añadir el trazo para las diferencias en el subintervalo
    fig.add_trace(go.Scatter(x=subset['Payment Date'][1:], y=diff_values,
                             mode='lines', name=f'Periodo de Tiempo {i+1}'))

    ####Añadir el trazo para los excesos sobre el umbral del percentil 90 (picos positivos)
    fig.add_trace(go.Scatter(x=excess_positive_dates, y=excess_positive_values,
                             mode='markers', name=f'Excesos Positivos Periodo de tiempo {i+1}',
                             marker=dict(color='red', size=8, symbol='circle')))

    #####Añadir el trazo para los excesos por debajo del umbral del percentil 10 (picos negativos)
    fig.add_trace(go.Scatter(x=excess_negative_dates, y=excess_negative_values,
                             mode='markers', name=f'Excesos Negativos Peridoo de tiempo negativo {i+1}',
                             marker=dict(color='blue', size=8, symbol='triangle-down')))

####Configurar el formato del eje x para las fechas
fig.update_xaxes(
    tickformat='%Y-%m-%d',
    tickmode='auto',
    nticks=10  #----------Ajusta según la cantidad de etiquetas de fecha que quieras mostrar
)

#####Añadir títulos y etiquetas
fig.update_layout(
    title='Grafico de ingresos y gastos excesivos extremos',
    xaxis_title='Fecha',
    yaxis_title='Diferencia de Valor',
    legend_title='Leyenda',
    width=1000,
    height=600
)

fig.show()

In [30]:
import numpy as np
import plotly.graph_objects as go

# Pedir al usuario que ingrese el número de subintervalo a visualizar
subinterval_index = int(input("Por favor, ingresa el número del subintervalo que quieres visualizar (ejemplo: 1, 2, 3...): "))

# Asegurarse de que el índice sea válido
if subinterval_index < 1 or subinterval_index > len(subsets):
    print("Índice fuera de rango. Elige un número entre 1 y", len(subsets))
else:
    # Obtener el subintervalo correspondiente (ajustando el índice porque Python comienza desde 0)
    subset = subsets[subinterval_index - 1]

    # Calcular diferencias
    diff_values = subset[' Amount'].diff().dropna()
    dates = subset['Payment Date'][1:]

    # Calcular percentiles
    threshold_90 = np.percentile(diff_values, 90)
    threshold_6 = np.percentile(diff_values, 6)

    # Identificar excesos positivos y negativos
    excess_positive_values = diff_values[diff_values > threshold_90]
    excess_positive_dates = dates[diff_values > threshold_90]

    excess_negative_values = diff_values[diff_values < threshold_6]
    excess_negative_dates = dates[diff_values < threshold_6]

    # Crear figura
    fig = go.Figure()

    # Línea de diferencias
    fig.add_trace(go.Scatter(x=dates, y=diff_values,
                             mode='lines', name=f'Diferencias Subintervalo {subinterval_index}'))

    # Picos positivos
    fig.add_trace(go.Scatter(x=excess_positive_dates, y=excess_positive_values,
                             mode='markers', name='Excesos Positivos (> P90)',
                             marker=dict(color='red', size=8, symbol='circle')))

    # Picos negativos
    fig.add_trace(go.Scatter(x=excess_negative_dates, y=excess_negative_values,
                             mode='markers', name='Excesos Negativos (< P6)',
                             marker=dict(color='blue', size=8, symbol='triangle-down')))

    # Añadir líneas horizontales para los percentiles
    fig.add_trace(go.Scatter(x=dates, y=[threshold_90] * len(dates), mode='lines',
                             name=f'Umbral promedio superior', line=dict(color='darkred', dash='dash')))

    fig.add_trace(go.Scatter(x=dates, y=[threshold_6] * len(dates), mode='lines',
                             name=f'Umbral promedio inferior', line=dict(color='blue', dash='dash')))

    # Eje X formato
    fig.update_xaxes(
        tickformat='%Y-%m-%d',
        tickmode='auto',
        nticks=10
    )

    # Etiquetas y layout
    fig.update_layout(
        title=f'Excesos de Ingresos/Gastos en el periodo de tiempo {subinterval_index}',
        xaxis_title='Fecha',
        yaxis_title='Diferencia de Valor',
        legend_title='Leyenda',
        width=1000,
        height=600
    )

    fig.show()

In [31]:
# Pedir al usuario que ingrese el número del subintervalo a visualizar
subinterval_index = int(input("Por favor, ingresa el número del subintervalo que quieres visualizar (ejemplo: 1, 2, 3...): "))

# Asegurarse de que el índice sea válido
if subinterval_index < 1 or subinterval_index > len(subsets):
    print("Índice fuera de rango. Elige un número entre 1 y", len(subsets))
else:
    # Obtener el subintervalo correspondiente (ajustando el índice porque Python comienza desde 0)
    subset = subsets[subinterval_index - 1]

    # Calcular diferencias
    diff_values = subset[' Amount'].diff().dropna()
    dates = subset['Payment Date'][1:]

    # Calcular percentiles
    threshold_90 = np.percentile(diff_values, 90)
    threshold_6 = np.percentile(diff_values, 6)

    # Identificar excesos positivos y negativos
    excess_positive_values = diff_values[diff_values > threshold_90]
    excess_positive_dates = dates[diff_values > threshold_90]

    excess_negative_values = diff_values[diff_values < threshold_6]
    excess_negative_dates = dates[diff_values < threshold_6]

    # Crear un DataFrame para los excesos
    excess_df = pd.DataFrame({
        'Fecha Excesos Positivos': excess_positive_dates,
        'Valor Excesos Positivos': excess_positive_values,
        'Fecha Excesos Negativos': excess_negative_dates,
        'Valor Excesos Negativos': excess_negative_values
    })

    # Crear figura
    fig = go.Figure()

    # Línea de diferencias
    fig.add_trace(go.Scatter(x=dates, y=diff_values,
                             mode='lines', name=f'Diferencias Subintervalo {subinterval_index}'))

    # Picos positivos
    fig.add_trace(go.Scatter(x=excess_positive_dates, y=excess_positive_values,
                             mode='markers', name='Excesos Positivos (> P90)',
                             marker=dict(color='red', size=8, symbol='circle')))

    # Picos negativos
    fig.add_trace(go.Scatter(x=excess_negative_dates, y=excess_negative_values,
                             mode='markers', name='Excesos Negativos (< P6)',
                             marker=dict(color='blue', size=8, symbol='triangle-down')))

    # Añadir líneas horizontales para los percentiles
    fig.add_trace(go.Scatter(x=dates, y=[threshold_90] * len(dates), mode='lines',
                             name=f'Percentil 90', line=dict(color='darkred', dash='dash')))

    fig.add_trace(go.Scatter(x=dates, y=[threshold_6] * len(dates), mode='lines',
                             name=f'Percentil 6', line=dict(color='blue', dash='dash')))

    # Eje X formato
    fig.update_xaxes(
        tickformat='%Y-%m-%d',
        tickmode='auto',
        nticks=10
    )

    # Etiquetas y layout
    fig.update_layout(
        title=f'Excesos de Ingresos/Gastos en Subintervalo {subinterval_index}',
        xaxis_title='Fecha',
        yaxis_title='Diferencia de Valor',
        legend_title='Leyenda',
        width=1000,
        height=600
    )

    fig.show()

In [32]:
excess_df.head()

,Fecha Excesos Positivos,Valor Excesos Positivos,Fecha Excesos Negativos,Valor Excesos Negativos
2402,2016-12-30,9117.90,NaT,NaN
2404,NaT,NaN,2017-01-05,-6800.0
2410,2017-01-09,3924.76,NaT,NaN
2431,2017-01-23,5765.10,NaT,NaN
2448,2017-01-31,5056.00,NaT,NaN


In [35]:
aux= np.array(excess_df['Valor Excesos Positivos'])
aux

array([ 9117.9 ,      nan,  3924.76,  5765.1 ,  5056.  ,  4700.  ,
        6500.  ,      nan, 28406.  ,      nan,  4667.  ,      nan,
        5074.  ,      nan, 10678.19,      nan,  7700.  ,  9000.  ,
            nan,  4247.73,  7151.1 ,      nan,  8788.  ,      nan,
        8750.  ,      nan,  9045.  ,      nan,  3872.  ,  6143.72,
            nan, 22056.  ,      nan,  9112.  ,      nan,  8671.92,
            nan,  7582.  ,  4150.  ])

In [36]:
excess_df.to_json('excess_example')